In [1]:
from google.cloud import storage

In [2]:
service_account_key_file =  '/Users/raghavendrakommavarapu/Documents/msds694_dist_computing/msds697-378721-8ba0b29dff82.json'

bucket_name = 'msds697_bucket'

In [16]:
storage_client = storage.Client.from_service_account_json(service_account_key_file)
bucket = storage_client.bucket(bucket_name)

##### first step: scrape all recipes

In [3]:
from scrape_all_recipes import *

In [4]:
all_recipes_api_url='https://www.allrecipes.com/servemodel/model.json?modelId=feedbacks&docId='
recipe_list_csv=''
recipe_index_pkl=''
recipe_data_json=''
n_recipes=50

In [6]:
url_list=['https://www.allrecipes.com/recipe/21176/baked-dijon-salmon/',
'https://www.allrecipes.com/recipe/146035/the-best-thai-coconut-soup/']

index = 0

In [7]:

recipe_data, failed_urls, new_list = all_recipes_scrape_recipe_page(url_list,index,all_recipes_api_url,n_recipes)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
50


In [9]:
recipe_data[0]

{'name': 'Baked Dijon Salmon Recipe',
 'url': 'https://www.allrecipes.com/recipe/21176/baked-dijon-salmon/',
 'publisher_url': 'https://www.allrecipes.com/cook/esmee',
 'publisher_name': 'Esmee Williams',
 'intro': '\nThis mustard-crusted salmon is a wonderful way to prepare fresh salmon fillets in the oven. Be sure to make extra — your family will be begging for more!',
 'nutrition': {'Calories': '422',
  'Servings': '4',
  'Total Fat': ('29g', '37%'),
  'Saturated Fat': ('10g', '51%'),
  'Cholesterol': ('97mg', '32%'),
  'Sodium': ('480mg', '21%'),
  'Total Carbohydrate': ('18g', '6%'),
  'Dietary Fiber': ('2g', '8%'),
  'Total Sugars': ('7g', ''),
  'Protein': ('24g', ''),
  'Vitamin C': ('27mg', '135%'),
  'Calcium': ('53mg', '4%'),
  'Iron': ('1mg', '7%'),
  'Potassium': ('500mg', '11%')},
 'directions': ['Preheat the oven to 400 degrees F (200 degrees C).',
  'Stir together butter, mustard, and honey in a small bowl. Set aside. Mix together bread crumbs, pecans, and parsley in an

In [10]:
len(recipe_data), len(failed_urls), len(new_list)

(48, 2, 161)

In [13]:
len(set(new_list))

161

In [17]:
# present files in GCS bucket
list(bucket.list_blobs())

[<Blob: msds697_bucket, recipe_data.csv, 1677195156551894>,
 <Blob: msds697_bucket, recipe_sample.json, 1677197023341278>,
 <Blob: msds697_bucket, tmp1, 1677197011607675>,
 <Blob: msds697_bucket, tmp2, 1677253310267785>,
 <Blob: msds697_bucket, tmp_text_file.txt, 1677196250804746>,
 <Blob: msds697_bucket, tmp_text_file3.txt, 1677253197056009>,
 <Blob: msds697_bucket, valid_books_ratings_tiny.csv, 1677194545957028>]

In [19]:
# update the recipe json on GCS

recipe_data_json_file = bucket.blob('recipe_data_json')

# get current contents
try:
    with recipe_data_json_file.open(mode='r') as f:
        recipe_data_curr=json.load(f)
except:
    recipe_data_curr = []

recipe_data_curr

[]

In [20]:

# append the present scrape and push to GCS
recipe_data_curr+=recipe_data

In [25]:
with recipe_data_json_file.open(mode='w') as f:
    json.dump(recipe_data_curr,f)


In [26]:
#  files in GCS bucket after push
list(bucket.list_blobs())

[<Blob: msds697_bucket, recipe_data.csv, 1677195156551894>,
 <Blob: msds697_bucket, recipe_data_json, 1677257505241779>,
 <Blob: msds697_bucket, recipe_sample.json, 1677197023341278>,
 <Blob: msds697_bucket, tmp1, 1677197011607675>,
 <Blob: msds697_bucket, tmp2, 1677253310267785>,
 <Blob: msds697_bucket, tmp_text_file.txt, 1677196250804746>,
 <Blob: msds697_bucket, tmp_text_file3.txt, 1677253197056009>,
 <Blob: msds697_bucket, valid_books_ratings_tiny.csv, 1677194545957028>]

In [27]:
# check after reading from GCS
with recipe_data_json_file.open(mode='r') as f:
    check_contents = json.load(f)

In [30]:
len(check_contents)

48

In [29]:
check_contents[0]

{'name': 'Baked Dijon Salmon Recipe',
 'url': 'https://www.allrecipes.com/recipe/21176/baked-dijon-salmon/',
 'publisher_url': 'https://www.allrecipes.com/cook/esmee',
 'publisher_name': 'Esmee Williams',
 'intro': '\nThis mustard-crusted salmon is a wonderful way to prepare fresh salmon fillets in the oven. Be sure to make extra — your family will be begging for more!',
 'nutrition': {'Calories': '422',
  'Servings': '4',
  'Total Fat': ['29g', '37%'],
  'Saturated Fat': ['10g', '51%'],
  'Cholesterol': ['97mg', '32%'],
  'Sodium': ['480mg', '21%'],
  'Total Carbohydrate': ['18g', '6%'],
  'Dietary Fiber': ['2g', '8%'],
  'Total Sugars': ['7g', ''],
  'Protein': ['24g', ''],
  'Vitamin C': ['27mg', '135%'],
  'Calcium': ['53mg', '4%'],
  'Iron': ['1mg', '7%'],
  'Potassium': ['500mg', '11%']},
 'directions': ['Preheat the oven to 400 degrees F (200 degrees C).',
  'Stir together butter, mustard, and honey in a small bowl. Set aside. Mix together bread crumbs, pecans, and parsley in an